In [1]:
import os
import glob
import psycopg2
import xmltodict
import json

from IPython.display import clear_output

BASE_DIR = '/home/pablo/base-lattes/Collection'

In [2]:
files_list = glob.glob("{}/*/*.xml".format(BASE_DIR))

n_files = len(files_list)
print('Arquivos XML Lattes: {}'.format(n_files))

Arquivos XML Lattes: 295303


In [3]:
dbConn = psycopg2.connect(
    host='127.0.0.1',
    user='cecilio',
    password='3.141592',
    dbname='lattes')

cursor = dbConn.cursor()

In [4]:
def curricula(doc, cursor, file):
    
    if 'CURRICULO-VITAE' in doc:

        if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
            id = file[35:-4].replace("/", "")
        elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
            id = file[35:-4].replace("/", "")
        else:
            id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']

        update_date = doc['CURRICULO-VITAE']['@DATA-ATUALIZACAO'] or None
        update_time = doc['CURRICULO-VITAE']['@HORA-ATUALIZACAO'] or None
        name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-COMPLETO'] or None
        citation_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS'] or None
        nationality = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NACIONALIDADE'] or None

        try:
            birth_place = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@PAIS-DE-NASCIMENTO'] or None
        except:
            birth_place = None

        try:
            birth_uf = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@UF-NASCIMENTO'] or None
        except:
            birth_uf = None 

        try:
            birth_city = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@CIDADE-NASCIMENTO'] or None          
        except:
            birth_city = None

        publicity = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@PERMISSAO-DE-DIVULGACAO'] or None

        try:
            nationality_abbrv = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@SIGLA-PAIS-NACIONALIDADE'] or None
        except:
            nationality_abbrv = None

        try:
            nationality_country = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@PAIS-DE-NACIONALIDADE'] or None
        except:
            nationality_country = None

        sql = "INSERT INTO curricula (id, update_date, update_time, name,\
               citation_name, nationality, birth_place, birth_uf, birth_city,\
               publicity, nationality_abbrv, nationality_country)\
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        values = (id, update_date, update_time, name, citation_name, nationality,
                  birth_place, birth_uf, birth_city, publicity, nationality_abbrv,
                  nationality_country)
        cursor.execute(sql, values)

In [5]:
def resumes(doc, cursor, file):

    if 'CURRICULO-VITAE' in doc:

        if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
            id = file[35:-4].replace("/", "")
        elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
            id = file[35:-4].replace("/", "")
        else:
            id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
            
        try:
            resume_pt = doc['CURRICULO-VITAE']['DADOS-GERAIS']['RESUMO-CV']['@TEXTO-RESUMO-CV-RH'] or None
        except:
            resume_pt = None
            
        try:
            resume_en = doc['CURRICULO-VITAE']['DADOS-GERAIS']['RESUMO-CV']['@TEXTO-RESUMO-CV-RH-EN'] or None
        except:
            resume_en = None
            
        sql = "INSERT INTO resumes (id, resume_pt, resume_en) VALUES (%s,%s,%s)"
        values = (id, resume_pt, resume_en)
        cursor.execute(sql, values)

In [6]:
def graduation(doc, cursor, file):

    try:
        graduation = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['GRADUACAO']
    except:
        return

    if isinstance(graduation, list):

        for grad in graduation:

            if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
                id = file[35:-4].replace("/", "")
            elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
                id = file[35:-4].replace("/", "")
            else:
                id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
                
            seq         = grad['@SEQUENCIA-FORMACAO'] or None
            level       = grad['@NIVEL'] or None

            cod_inst    = grad['@CODIGO-INSTITUICAO'] or None
            institution = grad['@NOME-INSTITUICAO'] or None
            cod_course  = grad['@CODIGO-CURSO'] or None
            course      = grad['@NOME-CURSO'] or None

            try:
                course_en = grad['@NOME-CURSO-INGLES'] or None
            except:
                course_en = None

            cod_area    = grad['@CODIGO-AREA-CURSO'] or None
            status      = grad['@STATUS-DO-CURSO'] or None
            start       = grad['@ANO-DE-INICIO'] or None
            close       = grad['@ANO-DE-CONCLUSAO'] or None

            try:
                monography = grad['@TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO'] or None
            except:
                monography = None

            sql = "INSERT INTO ac_bachelor (id, seq, level, cod_inst, institution, cod_course,\
                   course, course_en, cod_area, status, start, close, monography)\
                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (id, seq, level, cod_inst, institution, cod_course, course, course_en,
                      cod_area, status, start, close, monography)
            cursor.execute(sql, values)
            
    else:

        if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
            id = file[35:-4].replace("/", "")
        elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
            id = file[35:-4].replace("/", "")
        else:
            id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
            
        seq         = graduation['@SEQUENCIA-FORMACAO'] or None
        level       = graduation['@NIVEL'] or None

        cod_inst    = graduation['@CODIGO-INSTITUICAO'] or None
        institution = graduation['@NOME-INSTITUICAO'] or None
        cod_course  = graduation['@CODIGO-CURSO'] or None
        course      = graduation['@NOME-CURSO'] or None

        try:
            course_en = graduation['@NOME-CURSO-INGLES'] or None
        except:
            course_en = None

        cod_area    = graduation['@CODIGO-AREA-CURSO'] or None
        status      = graduation['@STATUS-DO-CURSO'] or None
        start       = graduation['@ANO-DE-INICIO'] or None
        close       = graduation['@ANO-DE-CONCLUSAO'] or None
        
        try:
            monography = graduation['@TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO'] or None
        except:
            monography = None

        sql = "INSERT INTO ac_bachelor (id, seq, level, cod_inst, institution, cod_course,\
               course, course_en, cod_area, status, start, close, monography)\
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        values = (id, seq, level, cod_inst, institution, cod_course, course, course_en,
                  cod_area, status, start, close, monography)
        cursor.execute(sql, values)

In [7]:
def master(doc, cursor, file):

    try:
        mastering = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['MESTRADO']
    except:
        return

    if isinstance(mastering, list):

        for master in mastering:

            if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
                id = file[35:-4].replace("/", "")
            elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
                id = file[35:-4].replace("/", "")
            else:
                id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
                
            seq         = master['@SEQUENCIA-FORMACAO'] or None
            level       = master['@NIVEL'] or None

            cod_inst    = master['@CODIGO-INSTITUICAO'] or None
            institution = master['@NOME-INSTITUICAO'] or None
            cod_course  = master['@CODIGO-CURSO'] or None
            course      = master['@NOME-CURSO'] or None

            try:
                course_en = master['@NOME-CURSO-INGLES'] or None
            except:
                course_en = None

            cod_area    = master['@CODIGO-AREA-CURSO'] or None
            status      = master['@STATUS-DO-CURSO'] or None
            start       = master['@ANO-DE-INICIO'] or None
            close       = master['@ANO-DE-CONCLUSAO'] or None

            try:
                dissertation = master['@TITULO-DA-DISSERTACAO-TESE'] or None
            except:
                dissertation = None
                
            try:
                advisor = master['@NUMERO-ID-ORIENTADOR'] or None
            except:
                advisor = None
            
            try:
                knowledge_1 = master['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-GRANDE-AREA-DO-CONHECIMENTO'] or None
                knowledge_2 = master['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-AREA-DO-CONHECIMENTO'] or None
                knowledge_3 = master['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-SUB-AREA-DO-CONHECIMENTO'] or None
                knowledge_4 = master['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-ESPECIALIDADE'] or None
            except:
                knowledge_1 = None
                knowledge_2 = None
                knowledge_3 = None
                knowledge_4 = None
            
            try:       
                activity_1 = master['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-1'] or None
                activity_2 = master['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-2'] or None
                activity_3 = master['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-3'] or None
            except:
                activity_1 = None
                activity_2 = None
                activity_3 = None

            sql = "INSERT INTO ac_master (id, seq, level, cod_inst, institution, cod_course,\
                   course, course_en, cod_area, status, start, close, dissertation, advisor,\
                   knowledge_1, knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2,\
                   activity_3) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,\
                   %s,%s,%s,%s,%s,%s,%s,%s)"
            values = (id, seq, level, cod_inst, institution, cod_course, course, course_en,
                      cod_area, status, start, close, dissertation, advisor, knowledge_1,
                      knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2, activity_3)
            cursor.execute(sql, values)
            
    else:

        if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
            id = file[35:-4].replace("/", "")
        elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
            id = file[35:-4].replace("/", "")
        else:
            id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
            
        seq         = mastering['@SEQUENCIA-FORMACAO'] or None
        level       = mastering['@NIVEL'] or None

        cod_inst    = mastering['@CODIGO-INSTITUICAO'] or None
        institution = mastering['@NOME-INSTITUICAO'] or None
        cod_course  = mastering['@CODIGO-CURSO'] or None
        course      = mastering['@NOME-CURSO'] or None

        try:
            course_en = mastering['@NOME-CURSO-INGLES'] or None
        except:
            course_en = None

        cod_area    = mastering['@CODIGO-AREA-CURSO'] or None
        status      = mastering['@STATUS-DO-CURSO'] or None
        start       = mastering['@ANO-DE-INICIO'] or None
        close       = mastering['@ANO-DE-CONCLUSAO'] or None
        
        try:
            dissertation = mastering['@TITULO-DA-DISSERTACAO-TESE'] or None
        except:
            dissertation = None
            
        try:
            advisor = mastering['@NUMERO-ID-ORIENTADOR'] or None
        except:
            advisor = None
        
        try:
            knowledge_1 = mastering['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-GRANDE-AREA-DO-CONHECIMENTO'] or None
            knowledge_2 = mastering['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-AREA-DO-CONHECIMENTO'] or None
            knowledge_3 = mastering['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-SUB-AREA-DO-CONHECIMENTO'] or None
            knowledge_4 = mastering['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-ESPECIALIDADE'] or None
        except:
            knowledge_1 = None
            knowledge_2 = None
            knowledge_3 = None
            knowledge_4 = None

        try:
            activity_1 = mastering['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-1'] or None
            activity_2 = mastering['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-2'] or None
            activity_3 = mastering['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-3'] or None
        except:
            activity_1 = None
            activity_2 = None
            activity_3 = None

        sql = "INSERT INTO ac_master (id, seq, level, cod_inst, institution, cod_course,\
               course, course_en, cod_area, status, start, close, dissertation, advisor,\
               knowledge_1, knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2,\
               activity_3) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,\
               %s,%s,%s,%s,%s,%s,%s,%s)"
        values = (id, seq, level, cod_inst, institution, cod_course, course, course_en,
                  cod_area, status, start, close, dissertation, advisor, knowledge_1,
                  knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2, activity_3)
        cursor.execute(sql, values)

In [8]:
def doctor(doc, cursor, file):

    try:
        doctor = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['DOUTORADO']
    except:
        return

    if isinstance(doctor, list):

        for doct in doctor:

            if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
                id = file[35:-4].replace("/", "")
            elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
                id = file[35:-4].replace("/", "")
            else:
                id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
                
            seq         = doct['@SEQUENCIA-FORMACAO'] or None
            level       = doct['@NIVEL'] or None

            cod_inst    = doct['@CODIGO-INSTITUICAO'] or None
            institution = doct['@NOME-INSTITUICAO'] or None
            
            try:
                cod_course = doct['@CODIGO-CURSO'] or None
            except:
                cod_course = None
            
            try:
                course = doct['@NOME-CURSO'] or None
            except:
                course = None

            try:
                course_en = doct['@NOME-CURSO-INGLES'] or None
            except:
                course_en = None

            try:
                cod_area = doct['@CODIGO-AREA-CURSO'] or None
            except:
                cod_area = None

            status      = doct['@STATUS-DO-CURSO'] or None
            start       = doct['@ANO-DE-INICIO'] or None
            
            try:
                close = doct['@ANO-DE-CONCLUSAO'] or None
            except:
                close = None

            try:
                dissertation = doct['@TITULO-DA-DISSERTACAO-TESE'] or None
            except:
                dissertation = None
            
            try:
                dissertation_en = doct['@TITULO-DA-DISSERTACAO-TESE-INGLES'] or None
            except:
                dissertation_en = None
                
            try:
                advisor = doct['@NUMERO-ID-ORIENTADOR'] or None
            except:
                advisor = None
            
            try:
                knowledge_1 = doct['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-GRANDE-AREA-DO-CONHECIMENTO'] or None
                knowledge_2 = doct['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-AREA-DO-CONHECIMENTO'] or None
                knowledge_3 = doct['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-SUB-AREA-DO-CONHECIMENTO'] or None
                knowledge_4 = doct['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-ESPECIALIDADE'] or None
            except:
                knowledge_1 = None
                knowledge_2 = None
                knowledge_3 = None
                knowledge_4 = None
                
            try:
                activity_1 = doct['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-1'] or None
                activity_2 = doct['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-2'] or None
                activity_3 = doct['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-3'] or None
            except:
                activity_1 = None
                activity_2 = None
                activity_3 = None
            
            try:       
                word_1 = doct['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-1'] or None
                word_2 = doct['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-2'] or None
                word_3 = doct['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-3'] or None
                word_4 = doct['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-4'] or None
                word_5 = doct['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-5'] or None
                word_6 = doct['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-6'] or None
            except:
                word_1 = None
                word_2 = None
                word_3 = None
                word_4 = None
                word_5 = None
                word_6 = None

            sql = "INSERT INTO ac_doctor (id, seq, level, cod_inst, institution, cod_course,\
                   course, course_en, cod_area, status, start, close, dissertation, dissertation_en,\
                   advisor, knowledge_1, knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2,\
                   activity_3, word_1, word_2, word_3, word_4, word_5, word_6) VALUES (%s,%s,%s,%s,%s,%s,\
                   %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (id, seq, level, cod_inst, institution, cod_course, course, course_en, cod_area,
                      status, start, close, dissertation, dissertation_en, advisor, knowledge_1,
                      knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2, activity_3,
                      word_1, word_2, word_3, word_4, word_5, word_6)
            cursor.execute(sql, values)
            
    else:

        if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
            id = file[35:-4].replace("/", "")
        elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
            id = file[35:-4].replace("/", "")
        else:
            id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
            
        seq         = doctor['@SEQUENCIA-FORMACAO'] or None
        level       = doctor['@NIVEL'] or None

        cod_inst    = doctor['@CODIGO-INSTITUICAO'] or None
        institution = doctor['@NOME-INSTITUICAO'] or None
        
        try:
            cod_course = doctor['@CODIGO-CURSO'] or None
        except:
            cod_course = None

        try:
            course = doctor['@NOME-CURSO'] or None
        except:
            course = None

        try:
            course_en = doctor['@NOME-CURSO-INGLES'] or None
        except:
            course_en = None
            
        try:
            cod_area = doctor['@CODIGO-AREA-CURSO'] or None
        except:
            cod_area = None

        status      = doctor['@STATUS-DO-CURSO'] or None
        start       = doctor['@ANO-DE-INICIO'] or None
        
        try:
            close = doctor['@ANO-DE-CONCLUSAO'] or None
        except:
            close = None
        
        try:
            dissertation = doctor['@TITULO-DA-DISSERTACAO-TESE'] or None
        except:
            dissertation = None

        try:
            dissertation_en = doctor['@TITULO-DA-DISSERTACAO-TESE-INGLES'] or None
        except:
            dissertation_en = None
            
        try:
            advisor = doctor['@NUMERO-ID-ORIENTADOR'] or None
        except:
            advisor = None
        
        try:
            knowledge_1 = doctor['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-GRANDE-AREA-DO-CONHECIMENTO'] or None
            knowledge_2 = doctor['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-AREA-DO-CONHECIMENTO'] or None
            knowledge_3 = doctor['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-SUB-AREA-DO-CONHECIMENTO'] or None
            knowledge_4 = doctor['AREAS-DO-CONHECIMENTO']['AREA-DO-CONHECIMENTO-1']['@NOME-DA-ESPECIALIDADE'] or None
        except:
            knowledge_1 = None
            knowledge_2 = None
            knowledge_3 = None
            knowledge_4 = None
            
        try:
            activity_1 = doctor['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-1'] or None
            activity_2 = doctor['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-2'] or None
            activity_3 = doctor['SETORES-DE-ATIVIDADE']['@SETOR-DE-ATIVIDADE-3'] or None
        except:
            activity_1 = None
            activity_2 = None
            activity_3 = None

        try:       
            word_1 = doctor['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-1'] or None
            word_2 = doctor['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-2'] or None
            word_3 = doctor['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-3'] or None
            word_4 = doctor['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-4'] or None
            word_5 = doctor['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-5'] or None
            word_6 = doctor['PALAVRAS-CHAVE']['@PALAVRA-CHAVE-6'] or None
        except:
            word_1 = None
            word_2 = None
            word_3 = None
            word_4 = None
            word_5 = None
            word_6 = None

        sql = "INSERT INTO ac_doctor (id, seq, level, cod_inst, institution, cod_course,\
               course, course_en, cod_area, status, start, close, dissertation, dissertation_en,\
               advisor, knowledge_1, knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2,\
               activity_3, word_1, word_2, word_3, word_4, word_5, word_6) VALUES (%s,%s,%s,%s,%s,%s,\
               %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        values = (id, seq, level, cod_inst, institution, cod_course, course, course_en, cod_area,
                  status, start, close, dissertation, dissertation_en, advisor, knowledge_1,
                  knowledge_2, knowledge_3, knowledge_4, activity_1, activity_2, activity_3,
                  word_1, word_2, word_3, word_4, word_5, word_6)
        cursor.execute(sql, values)

In [9]:
def articles(doc, cursor, file):

    try:
        articles = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
    except:
        return

    if isinstance(articles, list):

        for article in articles:
            
            if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
                id = file[35:-4].replace("/", "")
            elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
                id = file[35:-4].replace("/", "")
            else:
                id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
                
            seq = article['@SEQUENCIA-PRODUCAO'] or None
            
            try:
                value = article['@ORDEM-IMPORTANCIA'] or None
            except:
                value = None
                
            nature   = article['DADOS-BASICOS-DO-ARTIGO']['@NATUREZA'] or None
            title    = article['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'] or None

            try:
                title_en = article['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO-INGLES'] or None
            except:
                title_en = None

            year = article['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'] or None
            
            try:
                country = article['DADOS-BASICOS-DO-ARTIGO']['@PAIS-DE-PUBLICACAO'] or None
            except:
                country = None

            language = article['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] or None
            pub_env  = article['DADOS-BASICOS-DO-ARTIGO']['@MEIO-DE-DIVULGACAO'] or None
            
            try:
                url = article['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO'] or None
            except:
                url = None

            try:
                doi = article['DADOS-BASICOS-DO-ARTIGO']['@DOI'] or None
            except:
                doi = None
            
            try:
                periodic = article['DETALHAMENTO-DO-ARTIGO']['@TITULO-DO-PERIODICO-OU-REVISTA'] or None
            except:
                periodic = None

            try:
                p_local = article['DETALHAMENTO-DO-ARTIGO']['@LOCAL-DE-PUBLICACAO'] or None
            except:
                p_local = None

            try:
                issn = article['DETALHAMENTO-DO-ARTIGO']['@ISSN'] or None
            except:
                issn = None
            
            try:
                authors = article['AUTORES']

                coauthors = {}
                coauthor = []

                if isinstance(authors, list):

                    for author in authors:
                        try:
                            id_cnpq = author['@NRO-ID-CNPQ']
                        except:
                            id_cnpq = None

                        coauthor.append({"order" : author['@ORDEM-DE-AUTORIA'],
                                         "name" : author['@NOME-COMPLETO-DO-AUTOR'],
                                         "citation" : author['@NOME-PARA-CITACAO'],
                                         "id_cnpq" : id_cnpq})
                        coauthors['coauthors'] = coauthor

                else:

                    try:
                        id_cnpq = authors['@NRO-ID-CNPQ']
                    except:
                        id_cnpq = None

                    coauthor.append({"order" : authors['@ORDEM-DE-AUTORIA'],
                                     "name" : authors['@NOME-COMPLETO-DO-AUTOR'],
                                     "citation" : authors['@NOME-PARA-CITACAO'],
                                     "id_cnpq" : id_cnpq})
                    coauthors['coauthors'] = coauthor

                json_data = json.dumps(coauthors)

            except:
                json_data = None
                
            sql = "INSERT INTO articles (id, seq, value, nature, title, title_en, year, country,\
                   language, pub_env, url, doi, periodic, p_local, issn, coauthors)\
                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (id, seq, value, nature, title, title_en, year, country,language, pub_env,
                      url, doi, periodic, p_local, issn, json_data)
            cursor.execute(sql, values)
            
    else:

        if '@NUMERO-IDENTIFICADOR' not in doc['CURRICULO-VITAE']:
            id = file[35:-4].replace("/", "")
        elif doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'] == "":
            id = file[35:-4].replace("/", "")
        else:
            id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']

        seq = articles['@SEQUENCIA-PRODUCAO'] or None
        
        try:
            value = articles['@ORDEM-IMPORTANCIA'] or None
        except:
            value = None

        nature   = articles['DADOS-BASICOS-DO-ARTIGO']['@NATUREZA'] or None
        title    = articles['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'] or None

        try:
            title_en = articles['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO-INGLES'] or None
        except:
            title_en = None

        year = articles['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'] or None

        try:
            country = articles['DADOS-BASICOS-DO-ARTIGO']['@PAIS-DE-PUBLICACAO'] or None
        except:
            country = None

        language = articles['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] or None
        pub_env  = articles['DADOS-BASICOS-DO-ARTIGO']['@MEIO-DE-DIVULGACAO'] or None

        try:
            url = articles['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO'] or None
        except:
            url = None

        try:
            doi = articles['DADOS-BASICOS-DO-ARTIGO']['@DOI'] or None
        except:
            doi = None

        try:
            periodic = articles['DETALHAMENTO-DO-ARTIGO']['@TITULO-DO-PERIODICO-OU-REVISTA'] or None
        except:
            periodic = None

        try:
            p_local = articles['DETALHAMENTO-DO-ARTIGO']['@LOCAL-DE-PUBLICACAO'] or None
        except:
            p_local = None

        try:
            issn = articles['DETALHAMENTO-DO-ARTIGO']['@ISSN'] or None
        except:
            issn = None

        try:
            authors = articles['AUTORES']

            coauthors = {}
            coauthor = []

            if isinstance(authors, list):

                for author in authors:

                    try:
                        id_cnpq = author['@NRO-ID-CNPQ']
                    except:
                        id_cnpq = None

                    coauthor.append({"order" : author['@ORDEM-DE-AUTORIA'],
                                     "name" : author['@NOME-COMPLETO-DO-AUTOR'],
                                     "citation" : author['@NOME-PARA-CITACAO'],
                                     "id_cnpq" : id_cnpq})
                    coauthors['coauthors'] = coauthor

            else:

                try:
                    id_cnpq = authors['@NRO-ID-CNPQ']
                except:
                    id_cnpq = None

                coauthor.append({"order" : authors['@ORDEM-DE-AUTORIA'],
                                 "name" : authors['@NOME-COMPLETO-DO-AUTOR'],
                                 "citation" : authors['@NOME-PARA-CITACAO'],
                                 "id_cnpq" : id_cnpq})
                coauthors['coauthors'] = coauthor

            json_data = json.dumps(coauthors)
        
        except:
            json_data = None

        sql = "INSERT INTO articles (id, seq, value, nature, title, title_en, year, country,\
               language, pub_env, url, doi, periodic, p_local, issn, coauthors)\
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        values = (id, seq, value, nature, title, title_en, year, country,language, pub_env,
                  url, doi, periodic, p_local, issn, json_data)
        cursor.execute(sql, values)

In [ ]:
for idx, file in enumerate(files_list):

    with open(file, encoding="ISO-8859-1") as fd:
        doc = xmltodict.parse(fd.read())
        #curricula(doc, cursor, file)
        #resumes(doc, cursor, file)
        #graduation(doc, cursor, file)
        #master(doc, cursor, file)
        #doctor(doc, cursor, file)
        articles(doc, cursor, file)

        clear_output(wait=True)
        print('Processando... {:.2f}%'.format(idx * 100 / n_files))

Processando... 99.65%


In [ ]:
cursor.close()
dbConn.commit()
dbConn.close()